In [1]:
import requests
import json
import os
import sys
import time
import pandas as pd
import pickle
from utils import helper as hp
from utils.config_util import Config

## URLS

In [2]:
c = Config('configs/url_config.yaml')
URLS = c.config['URLS']
print(URLS)


{'MEDAL_URL': 'https://sph-c-api.olympics.com/summer/competition/api/ENG/medals', 'TEAM_PROFILE_URL': 'https://olympics.com/OG2024/data/MIS_Team~comp:OG2024~code:{code}~lang:ENG.json', 'EVENTS_URL': 'https://sph-i-api.olympics.com/summer/info/api/en/disciplinesevents', 'NOCS_URL': 'https://sph-i-api.olympics.com/summer/info/api/en/nocs', 'DISCIPLINES_URL': 'https://olympics.com/OG2024/data/GLO_Disciplines~comp=OG2024~lang=ENG.json', 'PARTICIPANTS_URL': 'https://olympics.com/OG2024/data/MIS_ParticipantNames~comp=OG2024~lang=ENG.json', 'ATHELETES_URL': 'https://sph-i-api.olympics.com/summer/info/api/ENG/athletes', 'MEDALS_URL': 'https://sph-c-api.olympics.com/summer/competition/api/ENG/medals', 'BASE_API_URL': 'https://sph-s-api.olympics.com/summer/schedules/api/ENG/schedule'}


In [3]:


NOCS_json = hp.fetch_json(URLS["NOCS_URL"])
NOCS_list = NOCS_json['nocs']
noc_df = pd.DataFrame.from_dict(NOCS_list)
noc_df = noc_df[['id', 'longName', 'continent']]
noc_df.rename(columns={'longName':'noc_name', 'id': 'noc_code'}, inplace=True)
noc_df.to_csv('csvs/noc.csv', index=False)



# Disciplines List

In [4]:
discipline_json = hp.fetch_json(URLS["DISCIPLINES_URL"])
disciplines = discipline_json['disciplines']
disciplines = [{'discipline_code': d['code'], 'discipline': d['description']} for d in disciplines if d['isSport'] ]
# save to simple csv file for future use
discipline_df = pd.DataFrame(disciplines)
discipline_df.to_csv('csvs/all_disciplines.csv', index=False)

https://olympics.com/OG2024/data/GLO_Disciplines~comp=OG2024~lang=ENG.json


In [5]:
_data = hp.fetch_json(URLS["PARTICIPANTS_URL"])
hp.save_json(_data, 'jsons/participants.json')
if _data:
    persons = _data['persons']
    file_list = os.listdir('data')
    file_list = [i.split('.')[0].strip() for i in file_list]
    print(file_list)
    cum = 1
    with open('error2.txt', 'w') as f:
        for i in persons:
            if cum % 100 == 0:
                print(cum)
            code = i['code']
            if code not in file_list:
                # print(code)
                try:
                    json_data = hp.fetch_json(f'https://olympics.com/OG2024/data/CIS_Bio_Athlete~comp=OG2024~code={code}~lang=ENG.json')
                    if json_data:
                        hp.save_json(json_data, f'data/{code}.json')
                        cum += 1
                except Exception as e:
                    f.write(f'{code} {str(e)}\n ')
            
    
            

['1930706', '1571488', '9251018', '9070188', '1944914', '1970548', '1926328', '1921117', '1565561', '9200447', '1895386', '1916264', '1974858', '9014699', '9330669', '1969769', '9240609', '1948034', '1901958', '1954017', '9163096', '1978528', '1953678', '9290141', '9440350', '9170340', '9360974', '1913436', '1550785', '9014733', '9330239', '1946883', '9200017', '9070022', '9100042', '1546651', '9430052', '9390223', '1961709', '1571522', '1542612', '9380673', '9360427', '1909191', '9250609', '9350091', '1894597', '1893252', '9331078', '9380389', '1934315', '9450700', '1880184', '9180544', '9018340', '9431213', '1571871', '9311837', '9210017', '9060022', '1557943', '9301564', '9110042', '1556651', '9420052', '9190817', '9380223', '9173096', '1570333', '9310375', '9450350', '1933080', '9430951', '1960118', '1920643', '9020283', '3539442', '1927186', '9040882', '9012608', '9360098', '9041790', '1921951', '9010635', '1898808', '9310660', '1903123', '9460709', '1950541', '9360562', '9380736'

## Make Atletes Profile

In [6]:

path_to_json = 'jsons/participants.json'
with open(path_to_json, 'r') as file:
    _data = json.load(file)
persons = _data['persons']
file_list = os.listdir('data')
file_list = [i.split('.')[0].strip() for i in file_list]
print(file_list)


['1930706', '1571488', '9251018', '9070188', '1944914', '1970548', '1926328', '1921117', '1565561', '9200447', '1895386', '1916264', '1974858', '9014699', '9330669', '1969769', '9240609', '1948034', '1901958', '1954017', '9163096', '1978528', '1953678', '9290141', '9440350', '9170340', '9360974', '1913436', '1550785', '9014733', '9330239', '1946883', '9200017', '9070022', '9100042', '1546651', '9430052', '9390223', '1961709', '1571522', '1542612', '9380673', '9360427', '1909191', '9250609', '9350091', '1894597', '1893252', '9331078', '9380389', '1934315', '9450700', '1880184', '9180544', '9018340', '9431213', '1571871', '9311837', '9210017', '9060022', '1557943', '9301564', '9110042', '1556651', '9420052', '9190817', '9380223', '9173096', '1570333', '9310375', '9450350', '1933080', '9430951', '1960118', '1920643', '9020283', '3539442', '1927186', '9040882', '9012608', '9360098', '9041790', '1921951', '9010635', '1898808', '9310660', '1903123', '9460709', '1950541', '9360562', '9380736'

In [7]:
codes = [i['code'] for i in persons]
codes = [i for i in codes if i in file_list]
l = []
code_2_disciplines = []
for code in codes:
    with open(f'data/{code}.json', 'r') as file:
        data = json.load(file)
        person_obj = data.get('person', {})
        mainfunction = data.get('mainfunction', {})
        if person_obj:
            mainfunction = person_obj.get('mainFunction', {})
            if mainfunction:
                mainfunc_desc = mainfunction.get('description', '').lower()
                
                if 'athlete' in mainfunc_desc:
                    
                    organisation = person_obj.get('organisation', {})
                    noc_code = organisation.get('code', 'NA')
                    noc_desc = organisation.get('description', 'NA')
                    
                    # nationality = person_obj.get('nationality', {})
                    # noc = nationality.get('code', 'NA')
                    # noc_full = nationality.get('description', 'NA')
                    person_gender = person_obj.get('personGender', {}).get('description', 'NA')
                    dob = person_obj.get('birthDate', 'NA')
                    
                    height = person_obj.get('height', 0)
                    weight = person_obj.get('weight', 0)
                    
                    given_name = person_obj.get('givenName', '')
                    family_name = person_obj.get('familyName', 'NA')
                    d = {}
                    d['mainfunc_desc'] = mainfunc_desc
                    d['code'] = code
                    d['noc'] = noc_code
                    d['noc_full'] = noc_desc
                    d['gender'] = person_gender
                    d['dob'] = dob
                    d['height'] = height
                    d['weight'] = weight
                    d['given_name'] = given_name
                    d['family_name'] = family_name
                    d['picture_url'] = f"'https://olympics.com/OG2024/pic/OG2024/00{str(code)[0]}/{str(code)[1:4]}/medium/{code}.png'"
                    
                    full_name = f'{given_name} {family_name}'.lower().strip()
                    formed_url = '-'.join(full_name.split())
                    formed_url = formed_url if given_name else '-' + formed_url
                    d['detail_url'] = f"'https://olympics.com/en/paris-2024/athlete/{formed_url}_{code}/'"
                    d['api_url'] = f"'https://olympics.com/OG2024/data/CIS_Bio_Athlete~comp=OG2024~code={code}~lang=ENG.json'"
                    if 'disciplines' in person_obj.keys():
                        l.append(d)
                        code_2_disciplines.extend([{'code': code, 'discipline_code': discipline['code'], 'discipline_desc': discipline['description']} for discipline in person_obj['disciplines']])
                
print(len(l))
print(len(code_2_disciplines))
    
        

11143
11175


In [8]:
athlete_df = pd.DataFrame(l)
athletes_disciplines_df = pd.DataFrame(code_2_disciplines)

for athlete_type in athlete_df['mainfunc_desc'].unique():
    athlete_df[athlete_df['mainfunc_desc'] == athlete_type].to_csv(f'csvs/{"_".join(athlete_type.split())}_athletes_comlete.csv', index=False)
athletes_disciplines_df.to_csv('csvs/athletes_disciplines_complete.csv', index=False)

In [9]:
athlete_df

,mainfunc_desc,code,noc,noc_full,gender,dob,height,weight,given_name,family_name,picture_url,detail_url,api_url
0,athlete,1532872,ARM,Armenia,Male,1991-10-21,0,0,Artur,Aleksanyan,'https://olympics.com/OG2024/pic/OG2024/001/53...,'https://olympics.com/en/paris-2024/athlete/ar...,'https://olympics.com/OG2024/data/CIS_Bio_Athl...
1,athlete,1532873,ARM,Armenia,Male,1999-01-22,0,0,Malkhas,Amoyan,'https://olympics.com/OG2024/pic/OG2024/001/53...,'https://olympics.com/en/paris-2024/athlete/ma...,'https://olympics.com/OG2024/data/CIS_Bio_Athl...
2,athlete,1532874,ARM,Armenia,Male,1996-12-21,0,0,Slavik,Galstyan,'https://olympics.com/OG2024/pic/OG2024/001/53...,'https://olympics.com/en/paris-2024/athlete/sl...,'https://olympics.com/OG2024/data/CIS_Bio_Athl...
3,athlete,1532944,ARM,Armenia,Male,1999-11-22,0,0,Arsen,Harutyunyan,'https://olympics.com/OG2024/pic/OG2024/001/53...,'https://olympics.com/en/paris-2024/athlete/ar...,'https://olympics.com/OG2024/data/CIS_Bio_Athl...
4,athlete,1532945,ARM,Armenia,Male,1999-10-27,0,0,Vazgen,Tevanyan,'https://olympics.com/OG2024/pic/OG2024/001/53...,'https://olympics.com/en/paris-2024/athlete/va...,'https://olympics.com/OG2024/data/CIS_Bio_Athl...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11138,athlete,4982175,TTO,Trinidad and Tobago,Male,2001-07-03,177,0,Elijah,Joseph,'https://olympics.com/OG2024/pic/OG2024/004/98...,'https://olympics.com/en/paris-2024/athlete/el...,'https://olympics.com/OG2024/data/CIS_Bio_Athl...
11139,athlete,4982762,UKR,Ukraine,Female,1999-01-31,0,0,Khrystyna,Homan,'https://olympics.com/OG2024/pic/OG2024/004/98...,'https://olympics.com/en/paris-2024/athlete/kh...,'https://olympics.com/OG2024/data/CIS_Bio_Athl...
11140,athlete,4983537,UGA,Uganda,Female,1993-07-29,174,0,Annet Chemengich,Chelangat,'https://olympics.com/OG2024/pic/OG2024/004/98...,'https://olympics.com/en/paris-2024/athlete/an...,'https://olympics.com/OG2024/data/CIS_Bio_Athl...
11141,athlete,4986655,GEQ,Equatorial Guinea,Female,2003-06-11,165,0,Sefora,Ada Eto,'https://olympics.com/OG2024/pic/OG2024/004/98...,'https://olympics.com/en/paris-2024/athlete/se...,'https://olympics.com/OG2024/data/CIS_Bio_Athl...


## Olympics Events

In [10]:
json_file_path = 'jsons/all_olympic_events.json'
reload_override = True


hp.initialize(URLS['BASE_API_URL'], reload_override, json_file_path)
# for all files I'll read its
all_data = []
print('reading page', json_file_path)
with open(json_file_path, 'r') as file:
    json_data = json.load(file)
print('json_data is ready!')
units = json_data['units']
groups = json_data['groups']
print('units and groups are ready!')
    

JSON data saved to jsons/all_olympic_events.json
reading page jsons/all_olympic_events.json
json_data is ready!
units and groups are ready!


## Build Modals CSVS

In [11]:
medals_json = hp.fetch_json(URLS["MEDALS_URL"])
hp.save_json(medals_json, 'jsons/medals.json')

medal_tables = medals_json['medalStandings']['medalsTable']
total_medals = [] 
_rank = []
for each in medal_tables:
    organisation = each['organisation']
    medals_number_list = each['medalsNumber']
    _rank.append(each['rank'])
    total_obj = [t for t in medals_number_list if t['type'] == 'Total'][0]
    # del total_obj['type']
    total_obj['organisation'] = organisation
    total_obj['rank'] = each['rank']
    total_medals.append(total_obj)
    
noc_df = pd.read_csv('csvs/noc.csv')
print(total_medals)
medals_df = pd.DataFrame(total_medals)
medals_df.drop(columns=['type'], inplace=True)
medals_df['country'] = medals_df['organisation'].apply(lambda x: noc_df[noc_df['noc_code'] == x]['noc_name'].values[0])
medals_df.to_csv('csvs/medals.csv', index=False)
medals_df


[{'type': 'Total', 'gold': 39, 'silver': 27, 'bronze': 24, 'total': 90, 'organisation': 'CHN', 'rank': 1}, {'type': 'Total', 'gold': 38, 'silver': 42, 'bronze': 42, 'total': 122, 'organisation': 'USA', 'rank': 2}, {'type': 'Total', 'gold': 18, 'silver': 18, 'bronze': 14, 'total': 50, 'organisation': 'AUS', 'rank': 3}, {'type': 'Total', 'gold': 18, 'silver': 12, 'bronze': 13, 'total': 43, 'organisation': 'JPN', 'rank': 4}, {'type': 'Total', 'gold': 16, 'silver': 24, 'bronze': 22, 'total': 62, 'organisation': 'FRA', 'rank': 5}, {'type': 'Total', 'gold': 14, 'silver': 22, 'bronze': 27, 'total': 63, 'organisation': 'GBR', 'rank': 6}, {'type': 'Total', 'gold': 13, 'silver': 8, 'bronze': 9, 'total': 30, 'organisation': 'KOR', 'rank': 7}, {'type': 'Total', 'gold': 13, 'silver': 7, 'bronze': 12, 'total': 32, 'organisation': 'NED', 'rank': 8}, {'type': 'Total', 'gold': 12, 'silver': 11, 'bronze': 8, 'total': 31, 'organisation': 'GER', 'rank': 9}, {'type': 'Total', 'gold': 11, 'silver': 13, 'bro

,gold,silver,bronze,total,organisation,rank,country
0,39,27,24,90,CHN,1,People's Republic of China
1,38,42,42,122,USA,2,United States of America
2,18,18,14,50,AUS,3,Australia
3,18,12,13,43,JPN,4,Japan
4,16,24,22,62,FRA,5,France
...,...,...,...,...,...,...,...
86,0,0,1,1,PUR,82,Puerto Rico
87,0,0,1,1,QAT,82,Qatar
88,0,0,1,1,SGP,82,Singapore
89,0,0,1,1,SVK,82,Slovakia
